In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 16.9 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [13]:
ngrok.kill()

In [14]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://subcritical-readily-mahalia.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://subcritical-readily-mahalia.ngrok-free.dev


True

In [15]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [16]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [9]:
result = stateful_query("簡介三星電子")
print(result)

三星電子（Samsung Electronics Co., Ltd.）是全球領先的科技公司，也是韓國三星集團旗下最大的核心子公司。公司於1969年成立，總部位於韓國京畿道水原市，行政總部則設於首爾瑞草區的三星城。

三星電子的業務範圍廣泛，主要涵蓋以下幾大領域：
*   **半導體業務**：是全球主要的半導體製造商之一，尤其在DRAM、NAND快閃記憶體、微控制器和影像感測器等領域居於產業領先地位，近年也積極投入微處理器和晶圓代工業務。三星電子在2009年曾超越英特爾，成為全球營收最大的半導體製造商，並持續在全球半導體市場佔據重要地位。此外，三星在高頻寬記憶體（HBM）等人工智慧晶片關鍵元件的研發也取得重要進展。
*   **行動通訊**：是全球最大的行動電話製造商之一，其Galaxy系列智慧型手機和平板電腦在全球市場上佔有領先地位。
*   **顯示器**：在顯示面板市場佔有重要份額，尤其在電視面板方面。三星顯示是市場上最大的面板廠之一，也生產QD-OLED面板。
*   **消費性電子產品**：生產各種家電產品，包括電視（如QLED電視）、冰箱、洗衣機、空調等。

三星電子在全球科技產業中擁有舉足輕重的地位，其產品銷往全球200多個國家。公司不僅是全球最大的消費電子產品、半導體和家用電器製造商之一，也在多個產品類別中保持全球第一的市佔率。三星電子在全球擁有龐大的員工隊伍和多個研發中心，持續透過創新引領科技發展。


In [10]:
result2 = stateful_query("最新旗艦手機是什麼？")
print(result2)

截至2026年1月4日，三星電子最新的旗艦手機系列是 **Samsung Galaxy S25 系列**。

該系列包含 Galaxy S25、Galaxy S25+ 和 Galaxy S25 Ultra 等型號，於2025年1月22日在美國聖荷西舉行的三星Galaxy Unpacked活動中發表，並於2025年2月7日上市。 其中，Galaxy S25 Ultra被許多評論認為是三星2025年度的旗艦機皇，在硬體規格和相機性能上都有頂尖的表現。

雖然目前已有關於2026年初將發表的Galaxy S26系列手機的傳聞和資訊，但它們尚未正式推出，因此Galaxy S25系列仍是目前三星電子已上市的最新旗艦機種。


In [17]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit


BODY:  {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"text","id":"595031214542029295","quoteToken":"kYrolt9CT2V379PaQGYCwB7_SYOlL5h-POUXKoTxDViibkw4caJq9VkupSC39OoOLyS155VGw3SXvHbkhHHUBQuFd-Yd9OiVMK2LnJc65xQtNIoYTx7p_hDLEUY4Rb0U4b37Obn3j1u5wi5tc4dd1Q","markAsReadToken":"W8dE4-eu3v4hWvmm0Q2Uv8vH6s5GGGDuR_Y_CQXfGU10W2_BjGASc6nME1DedfKsWu9ZnoXovo8i1AXJsxxWejTtBwWaN6rlMVJqQcbX4JVWZ3f3KoyHlpq1kGKcSh5csX2zrVNyjZQlNZucTeGT-7S3mFnHBXvTC40AULEVjVvCWn8XT33pDt6Ura2o-SPQiJBHEtHPkpVjLPsXPPS_5w","text":"AI 介紹一下三星電子(50字以內)"},"webhookEventId":"01KE3H0E4H69HX4KX1WA1NZX1N","deliveryContext":{"isRedelivery":false},"timestamp":1767497545845,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"831e3701a51f4ce6b6194294346c123e","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [04/Jan/2026 03:32:29] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"text","id":"595031239070057198","quoteToken":"jICVnYC4YxOdJld3vOyGokqoDFD2dRvi34knD2A362JkupHp2gl_KGTGLDODRw9_IElu6uB6NfcdA7fX5M2kQzAPUCTeJY4KITmVtxDLV-RwYjbUe3_Wo7RbrQ8n-0jx-dyG-bbrTCgKAoVuYviykQ","markAsReadToken":"PLKjqKxBlTKJ1iOm-PWRFi9sZKOkT6xkwW_YWfaqoQJOXol74ajbfPLSxiIfTysfeel6I0cHxDuTvM0vF0DqsMxp4MQ7ABC2Q1nUJF5SgJu20qcIgr0GOwBWCq0NPQU2fyZ6HTZgNlilbpNhjAfODFDsQwkFa8HCVDx0npH0LNbkbwrGaW2Xe1FPLp45vZXS1UM2cR3F64ESGFdCwarTLg","text":"AI 目前最新旗艦手機是什麼？"},"webhookEventId":"01KE3H0WZGFP958PDTNR08G6GQ","deliveryContext":{"isRedelivery":false},"timestamp":1767497560565,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"ec66acffd55b490dbc2c1420bc22b8e5","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [04/Jan/2026 03:32:46] "POST / HTTP/1.1" 200 -
